In [66]:
import pandas as pd
import numpy as np
import plotly.express as px

In [54]:
# Change these as needed
hashtag = "Dublin"
base_path = 'C:/Users/rohan/Documents/Text-Analytics-Lunar/results/'

lunar_phase_dict = {
    0: '1st Quarter',
    1: 'New Moon',
    2: '3rd Quarter',
    3: 'Full Moon'
}

# For dublin, may need to change for other locations

lunar_phase_date_dict = {
    '2019-12-26': 1, '2019-12-27': 1, '2019-12-28': 1, '2019-12-29': 1,
    '2019-12-30': 1, '2019-12-31': 1, '2020-01-01': 1, '2020-01-02': 1,
    '2020-01-03': 0, '2020-01-04': 0, '2020-01-05': 0, '2020-01-06': 0,
    '2020-01-07': 0, '2020-01-08': 0, '2020-01-09': 0, '2020-01-10': 3,
    '2020-01-11': 3, '2020-01-12': 3, '2020-01-13': 3, '2020-01-14': 3,
    '2020-01-15': 3, '2020-01-16': 3, '2020-01-17': 2, '2020-01-18': 2,
    '2020-01-19': 2, '2020-01-20': 2, '2020-01-21': 2, '2020-01-22': 2,
    '2020-01-23': 2, '2020-01-24': 1, '2020-01-25': 1, '2020-01-26': 1,
    '2020-01-27': 1, '2020-01-28': 1, '2020-01-29': 1, '2020-01-30': 1,
    '2020-01-31': 1, '2020-02-01': 1, '2020-02-02': 0, '2020-02-03': 0,
    '2020-02-04': 0, '2020-02-05': 0, '2020-02-06': 0, '2020-02-07': 0,
    '2020-02-08': 0, '2020-02-09': 3, '2020-02-10': 3, '2020-02-11': 3,
    '2020-02-12': 3, '2020-02-13': 3, '2020-02-14': 3, '2020-02-15': 2,
    '2020-02-16': 2, '2020-02-17': 2, '2020-02-18': 2, '2020-02-19': 2,
    '2020-02-20': 2, '2020-02-21': 2, '2020-02-22': 2, '2020-02-23': 1
}    

In [39]:
error_results = pd.read_csv("error_results.csv")

In [40]:
error_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Columns: 296 entries, filename to YOU_THING
dtypes: int64(295), object(1)
memory usage: 5.9+ MB


In [41]:
def parse_meta_data_from_filename(row):
    row['hashtag'] = row['filename'].replace(base_path, '').split('\\')[0]
    row['date'] = row['filename'].replace(base_path, '').split('\\')[1]
    row['file'] = row['filename'].split(row['date'] + '\\')[1].split('.')[0]
    row['lunar_phase'] = lunar_phase_date_dict[row['date']]
    return row


error_results = error_results.apply (lambda row: parse_meta_data_from_filename(row), axis=1)

In [42]:
error_results[['date', 'hashtag', 'file', 'error_count', 'lunar_phase']]

,date,hashtag,file,error_count,lunar_phase
0,2019-12-26,Dublin,0,0,1
1,2019-12-26,Dublin,1,4,1
2,2019-12-26,Dublin,10,1,1
3,2019-12-26,Dublin,11,2,1
4,2019-12-26,Dublin,12,0,1
...,...,...,...,...,...
2587,2020-02-23,Dublin,5,8,1
2588,2020-02-23,Dublin,6,0,1
2589,2020-02-23,Dublin,7,1,1
2590,2020-02-23,Dublin,8,2,1


In [69]:
error_counts_by_date_df = pd.DataFrame(columns=['date', 'hashtag', 'total_error_count', 'lunar_phase'])

for (key, value) in lunar_phase_date_dict.items():
    row = {
        'date': key,
        'lunar_phase': value,
        'hashtag': hashtag,
        'total_error_count': np.sum(error_results.loc[error_results['date'] == key]['error_count'])
    }
    
    error_counts_by_date_df = error_counts_by_date_df.append(row, ignore_index=True)

# error_counts_by_date_df['date'] = error_results['date'].astype('datetime64')

In [70]:
error_counts_by_date_df.head()

,date,hashtag,total_error_count,lunar_phase
0,2019-12-26,Dublin,44,1
1,2019-12-27,Dublin,68,1
2,2019-12-28,Dublin,92,1
3,2019-12-29,Dublin,114,1
4,2019-12-30,Dublin,36,1


In [72]:
fig = px.line(
    error_counts_by_date_df, 
    x="date", 
    y="total_error_count", 
    title='Total Error Counts over time for #{}'.format(hashtag)
)

fig.show()